In [2]:
!pip install kaggle unzip
!kaggle datasets download gauravduttakiit/loan-defaulter/
!unzip loan-defaulter

zsh:1: command not found: pip


100%|███████████████████████████████████████▊| 112M/112M [00:06<00:00, 18.9MB/s]
100%|████████████████████████████████████████| 112M/112M [00:06<00:00, 18.1MB/s]
Archive:  loan-defaulter.zip
  inflating: application_data.csv    
  inflating: columns_description.csv  
  inflating: previous_application.csv  


In [2]:
import pandas as pd
import numpy as np

application_data = pd.read_csv('application_data.csv')
previous_application = pd.read_csv('previous_application.csv')
columns_description = pd.read_csv(r'columns_description.csv',skiprows=1)

C:\Users\uzair\AppData\Local\Temp\ipykernel_20048\113819065.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
mergeddf = pd.merge(application_data,previous_application,on='SK_ID_CURR')

In [2]:
# split data
import pandas as pd
application_data = pd.read_csv('data/application_data.csv')
previous_application = pd.read_csv('data/previous_application.csv')
# columns_description = pd.read_csv(r'columns_description.csv',skiprows=1)

mergeddf =  pd.merge(application_data,previous_application,on='SK_ID_CURR')
data_A = mergeddf.sample(10000)
data_A.to_csv('data/loan_data_A.csv', index=False)

data_B = mergeddf.sample(10000)
data_B.to_csv('data/loan_data_B.csv', index=False)

data_C = mergeddf.sample(10000)
data_C.to_csv('data/loan_data_C.csv', index=False)


C:\Users\uzair\AppData\Local\Temp\ipykernel_4628\2925456544.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
mergeddf_sample = mergeddf.sample(10000)

In [7]:
# count rows
mergeddf_sample.shape[0]

10000

In [8]:
# drop columns with more than 50% missing values
mergeddf_sample = mergeddf_sample.dropna(thresh=0.5*len(mergeddf_sample), axis=1)

# convert categorical columns to numerical
mergeddf_sample = pd.get_dummies(mergeddf_sample)

# convert all columns to float
mergeddf_sample = mergeddf_sample.astype(float)


In [20]:

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler()),
])

# get features and labels. drop target column
X = mergeddf_sample.drop(['TARGET'],axis=1)
X = num_pipeline.fit_transform(X)
mergeddf_sample_num = num_pipeline.fit_transform(mergeddf_sample)

y = mergeddf_sample['TARGET']

# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state=42)

# Pytorch NN

In [22]:
import torch

model = torch.nn.Sequential(
            torch.nn.Linear(X_train.shape[1], 100),
            torch.nn.ReLU(),
            torch.nn.Linear(100, 50),
            torch.nn.ReLU(),
            torch.nn.Linear(50, 1),
            torch.nn.Sigmoid()
        )

# define loss function and optimizer
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# convert data to tensors
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.squeeze(torch.from_numpy(y_train.to_numpy()).float())
X_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.squeeze(torch.from_numpy(y_test.to_numpy()).float())

# train model
epochs = 10
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    # Forward pass
    y_pred = model(X_train_tensor).squeeze()
    # Compute Loss
    loss = criterion(y_pred, y_train_tensor)
    # Backward pass
    loss.backward()
    optimizer.step()
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))


Epoch 0: train loss: 0.731346070766449
Epoch 1: train loss: 0.7027109265327454
Epoch 2: train loss: 0.6751503944396973
Epoch 3: train loss: 0.6484552621841431
Epoch 4: train loss: 0.6224964261054993
Epoch 5: train loss: 0.5971705317497253
Epoch 6: train loss: 0.5723193883895874
Epoch 7: train loss: 0.5478552579879761
Epoch 8: train loss: 0.5237233638763428
Epoch 9: train loss: 0.49992331862449646


In [62]:
# accuracy score
from sklearn.metrics import accuracy_score
print(f'Accuracy score: {accuracy_score(y_test,y_pred)}')

# confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)



Accuracy score: 0.9125


array([[1817,   10],
       [ 165,    8]], dtype=int64)

In [ ]:
# store model weights with joblib